<a href="https://colab.research.google.com/github/Shrey-Viradiya/DiveDeep/blob/master/Prac6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prac 6
### 18BCE259

##### Processing Text with RNN

In [1]:
!nvidia-smi

Sun Apr 11 17:13:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    15W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# import logging
# logging.getLogger("tensorflow").setLevel(logging.ERROR)
# logging.getLogger("tensorflow").addHandler(logging.NullHandler(logging.ERROR))

In [4]:
data = " It is well known that medical diagnosis takes more time than deciding treatment for the condition .\n It might require plenty of time and skills of medical practitioners .\n In recent times, COVID-19 spread across the globe made computer researchers work toward faster and more accurate medical diagnosis solutions for hospitals .\n This project contains the code for diagnosing pneumonia from the chest X-ray images ."
text = data.split('\n')

In [5]:
text

[' It is well known that medical diagnosis takes more time than deciding treatment for the condition .',
 ' It might require plenty of time and skills of medical practitioners .',
 ' In recent times, COVID-19 spread across the globe made computer researchers work toward faster and more accurate medical diagnosis solutions for hospitals .',
 ' This project contains the code for diagnosing pneumonia from the chest X-ray images .']

In [6]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='!"#$*&()*+,-/:;<=>?@[\]^_{}~')
tokenizer.fit_on_texts(text)

In [7]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

53

In [8]:
sequences = tokenizer.texts_to_sequences(text)
l = len(sequences)

In [9]:
max_len = max([len(seq) for seq in sequences])
max_len

24

In [10]:
X = []
y = sequences
for sq in sequences:
    X.append(sq[:-1])

In [11]:
max_len = max([len(seq) for seq in sequences])

In [12]:
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=max_len+1)
y = tf.keras.preprocessing.sequence.pad_sequences(y, maxlen=max_len+1)
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)

In [13]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=10),
                                    tf.keras.layers.SimpleRNN(50, return_sequences=True),
                                    tf.keras.layers.Dense(vocab_size, activation="softmax")
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 10)          530       
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, None, 50)          3050      
_________________________________________________________________
dense (Dense)                (None, None, 53)          2703      
Total params: 6,283
Trainable params: 6,283
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(optimizer="RMSprop", loss="categorical_crossentropy", metrics=['accuracy'])
model.fit(X,y, epochs=200)

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 3.9621 - accuracy: 0.0100
Epoch 2/200
1/1 [==============================] - 0s 17ms/step - loss: 3.9178 - accuracy: 0.2700
Epoch 3/200
1/1 [==============================] - 0s 17ms/step - loss: 3.8719 - accuracy: 0.3100
Epoch 4/200
1/1 [==============================] - 0s 17ms/step - loss: 3.8126 - accuracy: 0.3500
Epoch 5/200
1/1 [==============================] - 0s 18ms/step - loss: 3.7437 - accuracy: 0.3100
Epoch 6/200
1/1 [==============================] - 0s 16ms/step - loss: 3.6752 - accuracy: 0.3800
Epoch 7/200
1/1 [==============================] - 0s 17ms/step - loss: 3.6472 - accuracy: 0.3400
Epoch 8/200
1/1 [==============================] - 0s 17ms/step - loss: 3.6620 - accuracy: 0.3400
Epoch 9/200
1/1 [==============================] - 0s 17ms/step - loss: 3.5746 - accuracy: 0.3400
Epoch 10/200
1/1 [==============================] - 0s 19ms/step - loss: 3.5098 - accuracy: 0.3600
Epoch 11/200
1/1 [===

In [15]:
def prob_of_input_sentence(model, tokenizer, sentence):
    print ("Input Sentence:", sentence)
    encoded=tokenizer.texts_to_sequences([sentence])[0]
    encoded.insert(0, 0)
    encoded=np.array(encoded)
    encoded=np.reshape(encoded, newshape=(1, -1))
    prob = model.predict(encoded, verbose=0)
    probability=1
    for i in range (prob. shape[1]-1):
        probability = probability * prob[0, i, encoded[0, i+1]]
    print("Probability: ", probability)

In [16]:
prob_of_input_sentence(model, tokenizer, "It is well known that")

Input Sentence: It is well known that
Probability:  7.032265161665041e-09


In [17]:
prob_of_input_sentence(model, tokenizer, "medical diagnosis takes more time than deciding")

Input Sentence: medical diagnosis takes more time than deciding
Probability:  7.808005211037956e-23


In [18]:
prob_of_input_sentence(model, tokenizer, "It is well known that medical diagnosis takes more time than deciding treatment for the condition")

Input Sentence: It is well known that medical diagnosis takes more time than deciding treatment for the condition
Probability:  6.447985054390805e-33


In [19]:
prob_of_input_sentence(model, tokenizer, "It might require plenty of time")

Input Sentence: It might require plenty of time
Probability:  4.248226324160745e-11


#### Predicting Next Chars

In [20]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [21]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [22]:
tokenizer.texts_to_sequences(['First'])

[[20, 6, 9, 8, 3]]

In [23]:
tokenizer.sequences_to_texts([[20,6,9,8,3]])

['f i r s t']

In [24]:
max_id = len(tokenizer.word_index)

In [25]:
max_id

39

In [26]:
dataset_size = tokenizer.document_count

In [27]:
dataset_size

1115394

In [28]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

### How to Split a Sequential Dataset

Let’s take the first 90% of the text for the training set (keeping the rest for the validation set and the test set), and create a tf.data.Dataset that will return each character one by one from this set:

In [29]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

### Chopping the Sequential Dataset into Multiple Windows

The training set now consists of a single sequence of over a million characters, so we can’t just train the neural network directly on it: the RNN would be equivalent to a deep net with over a million layers, and we would have a single (very long) instance to train it. Instead, we will use the dataset’s window() method to convert this long sequence of characters into many smaller windows of text. Every instance in the dataset will be a fairly short substring of the whole text, and the RNN will be unrolled only over the length of these substrings. This is called truncated backpropagation through time. 

In [30]:
n_steps = 100
window_length = n_steps + 1

In [31]:
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

The window() method creates a dataset that contains windows, each of which is also represented as a dataset. It’s a nested dataset, analogous to a list of lists. This is useful when you want to transform each window by calling its dataset methods (e.g., to shuffle them or batch them). However, we cannot use a nested dataset directly for training, as our model will expect tensors as input, not datasets. So, we must call the flat_map() method: it converts a nested dataset into a flat dataset (one that does not contain datasets).

In [32]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

We need to shuffle these windows. Then we can batch the windows and separate the inputs (the first 100 characters) from the target (the last character):

In [33]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

Categorical input features should generally be encoded, usually as one-hot vectors or as embeddings. Here, we will encode each character using a one-hot vector because there are fairly few distinct characters (only 39):

In [34]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [35]:
dataset = dataset.prefetch(1)

### Building and Training the Char-RNN Model

In [36]:
import os

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [37]:
class NvidiaUtilizationCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        text = !nvidia-smi
        text = text[9][60:65] + ' GPU utilization'
        print(text)

In [38]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation='softmax'))    
])

model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam')
steps_per_epoch = train_size // batch_size // n_steps

history = model.fit(dataset, epochs = 40, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback, NvidiaUtilizationCallback()])

Epoch 1/40
  0%  GPU utilization
313/313 [==============================] - 7s 8ms/step - loss: 2.9229
Epoch 2/40
 41%  GPU utilization
313/313 [==============================] - 3s 8ms/step - loss: 2.1152
Epoch 3/40
 41%  GPU utilization
313/313 [==============================] - 3s 8ms/step - loss: 1.8883
Epoch 4/40
 37%  GPU utilization
313/313 [==============================] - 3s 8ms/step - loss: 1.7260
Epoch 5/40
 34%  GPU utilization
313/313 [==============================] - 3s 8ms/step - loss: 1.5592
Epoch 6/40
 32%  GPU utilization
313/313 [==============================] - 3s 8ms/step - loss: 1.3938
Epoch 7/40
 25%  GPU utilization
313/313 [==============================] - 3s 8ms/step - loss: 1.3162
Epoch 8/40
 39%  GPU utilization
313/313 [==============================] - 3s 8ms/step - loss: 1.2756
Epoch 9/40
 39%  GPU utilization
313/313 [==============================] - 3s 8ms/step - loss: 1.2787
Epoch 10/40
 39%  GPU utilization
313/313 [==============================

### Using the Model to Generate Text

In [39]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [40]:
X_new = preprocess(["How are yo"])

In [41]:
Y_pred = np.argmax(model.predict(X_new), axis=-1)

In [42]:
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

'u'

### Generating Fake Shakespearean Text

In [43]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [44]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [45]:
text_1 = complete_text("t", temperature=0.2)

In [46]:
print(text_1)

theren the base court? sushich'd thou the king shal


In [47]:
text_2 = complete_text("w", temperature=1)

In [48]:
print(text_2)

wording to bid my lie.

king richard ii:
then let m


In [49]:
text_3 = complete_text("i", temperature=2)

In [50]:
print(text_3)

i emisb
abwell
a liee-nil, busso the caw;
sedt;
pre


In [51]:
text_4 = complete_text("I shall love", temperature=0.5, n_chars=100)

In [52]:
print(text_4)

I shall love, and his happling's rowards the propains, i speak thou hath well your heart destanders,
that evers 


In [53]:
text_5 = complete_text("love", temperature=1, n_chars = 150)

In [54]:
print(text_5)

lovedom and from me.
i wear, they he as throow his light is jusce have stiest hath
he blood'st may knore in that now with my hards a gows he wort make hea
